In [33]:
import pandas as pd
import numpy as np
fn = '../src/data/raw/Community_2005-2017_Greenhouse_Gas_Emissions.csv'
emissions = pd.read_csv(fn)
emissions_trunc = pd.DataFrame(data=emissions.iloc[0:12])
emissions_trunc = pd.concat([emissions_trunc, emissions.iloc[13:14]], ignore_index=True)
emissions_trunc.index = emissions_trunc['Unnamed Column'].to_list()
emissions_trunc.index.names = ['Source']
bad_char = [',',r'[\(\)]']
emissions_trunc = emissions_trunc.replace(bad_char,'', regex=True).fillna(value=0)
emissions_trunc = emissions_trunc.apply(pd.to_numeric, errors='coerce')
emissions_trunc.iloc[1]= (
    emissions_trunc.iloc[1] + 
    emissions_trunc.iloc[2] + 
    emissions_trunc.iloc[3] + 
    emissions_trunc.iloc[9])
#non-res electricity includes large and small; and gov-non-city-owned electricity; and CTA rail elec
emissions_trunc.iloc[5]= (emissions_trunc.iloc[5] + 
                          emissions_trunc.iloc[6] + 
                          emissions_trunc.iloc[7])
#non-res gas includes commercial, industrial, fugitive
emissions_trunc.iloc[11] = emissions_trunc.iloc[11]*-1
small_row_names = ['Large Commercial Electricity', 
                   'Commercial Gas Consumption', 
                   'Fugitive Gas Emissions', 
                   'CTA Rail Electricty']
emissions_trunc = emissions_trunc.drop(small_row_names)
rename_dict = {'Small Commercial Electricity':'Non-Residential Electricity',
              'Industrial Gas Consumption': 'Non-Residential Gas Consumption'}
emissions_trunc = emissions_trunc.rename(rename_dict).iloc[:,1:]
emissions_trunc.to_csv('../src/data/interim/2005-2017_lessTruncated.csv')

trunc_emissions = pd.read_csv('../src/data/interim/2005-2017_lessTruncated.csv')
year_str = [str(int(blah)) for blah in np.linspace(2005, 2017, 13)]
idx_list = trunc_emissions.index
non_totals = trunc_emissions['Source'].copy()
type_list = [list(non_totals.values),
             [non_totals.str.split(' ')[i][-1].strip() for i in idx_list],
             [non_totals.str.split(' ')[i][0].strip() for i in idx_list]]
idx = pd.MultiIndex.from_arrays(type_list, names=['original', 'source', 'type'])
len_columns = [len(trunc_emissions.columns)]
emissions_lvld = trunc_emissions.copy()
emissions_lvld.index = idx
emissions_lvld.loc[:, 'id'] = np.array(np.arange(len(emissions_lvld)))
emissions_lvld.loc[:, 'source'] = type_list[0]
emissions_lvld.loc[:, 'type'] = [
    'Electricity', 
    'Electricity', 
    'Electricity',
    'Gas',
    'Gas',
    'Vehicle',
    'Waste',
    'Renewable Energy Credit',
    'Municipal Mixed'
]
emissions_lvld = emissions_lvld.sort_index()
emissions_long = emissions_lvld.melt(col_level=0, id_vars=['id', 'source', 'type'], value_vars=year_str, var_name='Year')
emissions_long.sort_index()
emissions_long['id'] = emissions_long.index
emissions_long.rename(columns={'value':'MTCO2e', 'type':'Type'}, inplace=True)
emissions_long

,id,source,Type,Year,MTCO2e
0,0,"Community Waste (garbage, recycling and organics)",Waste,2005,17000.0
1,1,Estimated Municipal Operations GHG Emissions,Municipal Mixed,2005,24559.0
2,2,Government non-City-owned Electricty,Electricity,2005,641.0
3,3,Non-Residential Electricity,Electricity,2005,385451.0
4,4,Non-Residential Gas Consumption,Gas,2005,225283.0
5,5,Renewable Energy Credits (RECs),Renewable Energy Credit,2005,NaN
6,6,Residential Electricity,Electricity,2005,154524.0
7,7,Residential Gas Consumption,Gas,2005,114674.0
8,8,Vehicle Miles Traveled (VMT),Vehicle,2005,121736.0
9,9,"Community Waste (garbage, recycling and organics)",Waste,2006,17000.0


In [34]:
emissions_long.to_csv('../src/data/interim/emissions2017_long_simple.csv')

In [45]:
diff = emissions_trunc.diff(axis=1)
diff.to_csv('../src/data/interim/2005-2017_lessTruncated_diff.csv')
trunc_emissions = pd.read_csv('../src/data/interim/2005-2017_lessTruncated_diff.csv')
trunc_emissions = trunc_emissions.drop('2005', axis=1)
trunc_emissions

,Source,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Residential Electricity,-5484.0,1360.0,-3280.0,-11289.0,13981.0,-1863.0,5541.0,-13191.0,-657.0,-18234.0,-7073.0,-3239.0
1,Non-Residential Electricity,13756.0,6120.0,-11242.0,-13620.0,17388.0,-7245.0,2708.0,-11636.0,-9276.0,-32517.0,-39216.0,458.0
2,Government non-City-owned Electricty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0
3,Residential Gas Consumption,2117.0,2353.0,8151.0,-8533.0,-4045.0,5353.0,0.0,2388.0,-8136.0,4111.0,-12362.0,3418.0
4,Non-Residential Gas Consumption,-2313.0,13042.0,10693.0,-27021.0,-20223.0,12833.0,0.0,10272.0,-35670.0,35798.0,-9157.0,375.0
5,Vehicle Miles Traveled (VMT),0.0,0.0,0.0,0.0,-3652.0,0.0,0.0,0.0,0.0,472.0,3592.0,20777.0
6,"Community Waste (garbage, recycling and organics)",0.0,0.0,0.0,0.0,709.0,0.0,0.0,0.0,0.0,-1483.0,1990.0,259.0
7,Renewable Energy Credits (RECs),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-54582.0,-13458.0,16574.0,8849.0,7514.0
8,Estimated Municipal Operations GHG Emissions,0.0,0.0,0.0,0.0,-5308.0,0.0,0.0,0.0,0.0,-8562.0,-2545.0,1.0


In [50]:
year_str = [str(int(blah)) for blah in np.linspace(2006, 2017, 12)]
idx_list = trunc_emissions.index
non_totals = trunc_emissions['Source'].copy()
type_list = [list(non_totals.values),
             [non_totals.str.split(' ')[i][-1].strip() for i in idx_list],
             [non_totals.str.split(' ')[i][0].strip() for i in idx_list]]
idx = pd.MultiIndex.from_arrays(type_list, names=['original', 'source', 'type'])
len_columns = [len(trunc_emissions.columns)]
emissions_lvld = trunc_emissions.copy()
emissions_lvld.index = idx
emissions_lvld.loc[:, 'id'] = np.array(np.arange(len(emissions_lvld)))
emissions_lvld.loc[:, 'source'] = type_list[0]
emissions_lvld.loc[:, 'type'] = [
    'Electricity', 
    'Electricity', 
    'Electricity',
    'Gas',
    'Gas',
    'Vehicle',
    'Waste',
    'Renewable Energy Credit',
    'Municipal Mixed'
]
emissions_lvld = emissions_lvld.sort_index()
emissions_long = emissions_lvld.melt(col_level=0, id_vars=['id', 'source', 'type'], value_vars=year_str, var_name='Year')
emissions_long.sort_index()
emissions_long['id'] = emissions_long.index
emissions_long.rename(columns={'value':'dMTCO2e', 'type':'Type'}, inplace=True)
emissions_long

,id,source,Type,Year,dMTCO2e
0,0,"Community Waste (garbage, recycling and organics)",Waste,2006,0.0
1,1,Estimated Municipal Operations GHG Emissions,Municipal Mixed,2006,0.0
2,2,Government non-City-owned Electricty,Electricity,2006,0.0
3,3,Non-Residential Electricity,Electricity,2006,13756.0
4,4,Non-Residential Gas Consumption,Gas,2006,-2313.0
5,5,Renewable Energy Credits (RECs),Renewable Energy Credit,2006,NaN
6,6,Residential Electricity,Electricity,2006,-5484.0
7,7,Residential Gas Consumption,Gas,2006,2117.0
8,8,Vehicle Miles Traveled (VMT),Vehicle,2006,0.0
9,9,"Community Waste (garbage, recycling and organics)",Waste,2007,0.0


In [51]:
emissions_long.to_csv('../src/data/interim/emissions2017_long_diff_simple.csv')